In [ ]:
# Goal: semi-manually collect profile images & titles
# ./occupations.txt:
#     0,Engineer
#     1,QA
#     ...
# ./images/0.jpg
#   ...
# ./images/999.jpg
occupations_file = 'occupations.txt'
images_path = 'images/'

In [64]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [65]:
from selenium import webdriver

In [66]:
driver = webdriver.Chrome()
driver.implicitly_wait(5)

In [67]:
driver.get('https://www.linkedin.com/m/login/')

In [ ]:
# you log in here

In [68]:
driver.get('https://www.linkedin.com/mynetwork/invite-connect/connections/')

In [72]:
header = driver.find_element_by_class_name("mn-connections__header")
# parse stuff like'552 Connections', you may need to tab back here
expected_connections_count = int(header.text.split(' ')[0])
expected_connections_count

552

In [74]:
def scroll_until_all_connections_are_visible():
    import time
    elements = []
    prev_count = 0
    while len(elements) < expected_connections_count:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)");
        elements = driver.find_elements_by_class_name('mn-connection-card__occupation')
        current_count = len(elements)
        print(f"Can see {current_count} connections out of {expected_connections_count}")
        if (current_count < expected_connections_count):
            time.sleep(5)
        if (prev_count == current_count):
            print("Looks like we're stuck. You might want to switch to that browser for some focus and run this thing again")
            return

        prev_count = current_count

scroll_until_all_connections_are_visible()

Can see 120 connections out of 552
Can see 160 connections out of 552
Can see 200 connections out of 552
Can see 240 connections out of 552
Can see 280 connections out of 552
Can see 320 connections out of 552
Can see 359 connections out of 552
Can see 399 connections out of 552
Can see 437 connections out of 552
Can see 476 connections out of 552
Can see 515 connections out of 552
Can see 547 connections out of 552
Can see 547 connections out of 552
Looks like we're stuck. You might want to switch to that browser for some focus and run this thing again


In [82]:
elements = driver.find_elements_by_class_name('mn-connection-card')
len(elements)

547

In [83]:
def scroll_to(element):
    driver.execute_script("arguments[0].scrollIntoView();", element)

In [90]:
def extract_occupation(elements, idx):
    scroll_to(elements[idx])
    occupation = elements[idx].find_element_by_class_name('mn-connection-card__occupation').text
    with open(occupations_file, 'a') as f:
        f.write(f'{idx},{occupation}\n')

extract_occupation(elements, 0)

In [91]:
import os
os.remove(occupations_file)

In [107]:
import requests
def download_image(elements, idx):
    style = elements[idx].find_element_by_class_name('presence-entity__image').get_attribute('style')
    if (style == ''):
        print('Empty style (no img download) for: ' + elements[idx].text[:5])
        return

    image_url = style.split('"')[1]
    response = requests.get(image_url)
    if response.status_code != 200:
        raise ValueError("Something went wrong " + str(response))
        
    img = response.raw.read()
    with open(images_path + str(idx) + '.jpg', 'wb') as f:
        for chunk in response.iter_content(1024):
            f.write(chunk)

occupations = []
# limit the ids for ease of debugging, restarts are idempotent
# you might have to alt-tab back from time to time or keep the focus in that window until the very end
for idx, element in list(enumerate(elements))[400:600]:
    print(f"Extracting {idx}")
    scroll_to(elements[idx])
    download_image(elements, idx)
    extract_occupation(elements, idx)

Extracting 400
Empty style (no img download) for: Anton
Extracting 401
Extracting 402
Extracting 403
Extracting 404
Extracting 405
Extracting 406
Extracting 407
Extracting 408
Extracting 409
Extracting 410
Extracting 411
Extracting 412
Extracting 413
Extracting 414
Extracting 415
Extracting 416
Extracting 417
Extracting 418
Extracting 419
Extracting 420
Extracting 421
Extracting 422
Extracting 423
Extracting 424
Extracting 425
Extracting 426
Extracting 427
Extracting 428
Extracting 429
Extracting 430
Extracting 431
Extracting 432
Extracting 433
Extracting 434
Extracting 435
Extracting 436
Extracting 437
Extracting 438
Extracting 439
Extracting 440
Extracting 441
Extracting 442
Extracting 443
Extracting 444
Extracting 445
Extracting 446
Extracting 447
Extracting 448
Extracting 449
Extracting 450
Extracting 451
Extracting 452
Extracting 453
Empty style (no img download) for: Serge
Extracting 454
Empty style (no img download) for: Alexe
Extracting 455
Extracting 456
Extracting 457
Extract

In [108]:
# run this in case of exceptions 
%debug

> <ipython-input-99-b18af7b469b7>(7)download_image()
      5     if (style == ''):
      6         print('Empty style for ' + elements[idx].text)
----> 7     image_url = style.split('"')[1]
      8     response = requests.get(image_url)
      9     if response.status_code != 200:

ipdb> cont
